In [58]:
from collections import Counter
import numpy as np

In [47]:
p0 = np.array([0.12, 0.08, 0.56, 0.02, 0.22])
P = np.array([0.37,0.22,0.17,0.15,0.09,
0.25,0.13,0.14,0.11,0.37,
0.12,0.18,0.35,0.10,0.25,
0.11,0.07,0.10,0.47,0.25,
0.15,0.4,0.24,0.17,0.04]).reshape((5,5))
P

array([[ 0.37,  0.22,  0.17,  0.15,  0.09],
       [ 0.25,  0.13,  0.14,  0.11,  0.37],
       [ 0.12,  0.18,  0.35,  0.1 ,  0.25],
       [ 0.11,  0.07,  0.1 ,  0.47,  0.25],
       [ 0.15,  0.4 ,  0.24,  0.17,  0.04]])

In [57]:
class MarcovChain:
    def __init__(self, p0, P, state_names = None):
        if P.shape[0] != P.shape[1] != len(p0):
            raise "wrong shapes"
        self.p0 = p0
        self.P  = P
        self.n_states = len(p0)
        self.state_names = state_names
        
    def generate_path(self,size):
        current_state = np.random.choice(self.n_states, p = self.p0)
        path = [current_state]
        for i in range(size- 1):
            current_state = np.random.choice(self.n_states, p = self.P[:,current_state])
            path.append(current_state)
        return path if self.state_names is None else list(map(lambda x: self.state_names[x], path))

In [20]:
mc = MarcovChain(
    p0=p0,
    P = P,
    state_names = ["state1", "state2", "state3", "state4", "state5"]
)

In [21]:
l = mc.generate_path(365)
l[:10]

['state1',
 'state1',
 'state2',
 'state3',
 'state5',
 'state4',
 'state4',
 'state4',
 'state4',
 'state1']

In [22]:
Counter(l)

Counter({'state1': 65, 'state2': 67, 'state3': 82, 'state4': 83, 'state5': 68})

In [23]:
simulation = [mc.generate_path(365) for i in range(1000)]

In [25]:
last_states = list(map(lambda x: x[-1], simulation))

In [26]:
c=Counter(last_states)
c

Counter({'state1': 232,
         'state2': 191,
         'state3': 173,
         'state4': 212,
         'state5': 192})

In [27]:
np.array(list(c.values()))/1000

array([ 0.173,  0.192,  0.232,  0.212,  0.191])

In [48]:
A = P.T - np.identity(P.shape[0]) 
A = np.vstack([A,np.ones((1,P.shape[0]))])
A

array([[-0.63,  0.25,  0.12,  0.11,  0.15],
       [ 0.22, -0.87,  0.18,  0.07,  0.4 ],
       [ 0.17,  0.14, -0.65,  0.1 ,  0.24],
       [ 0.15,  0.11,  0.1 , -0.53,  0.17],
       [ 0.09,  0.37,  0.25,  0.25, -0.96],
       [ 1.  ,  1.  ,  1.  ,  1.  ,  1.  ]])

In [41]:
b = np.zeros(P.shape[0]+1)
b[-1] = 1
b

array([ 0.,  0.,  0.,  0.,  0.,  1.])

In [56]:
np.linalg.lstsq(A,b)

(array([ 0.2,  0.2,  0.2,  0.2,  0.2]),
 array([  3.42266081e-33]),
 5,
 array([ 2.23606798,  1.3362108 ,  0.89994706,  0.76455193,  0.64474516]))